In [1]:
import glob
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler

# 폴더 경로 지정
folder_path = r'C:\Users\pc\Desktop\CNN\CIC-AndMal2017'

In [2]:
# 랜섬웨어 클래스별 파일에 있는 샘플 개수를 확인하기 위한 딕셔너리
samples_per_class = {}

# Ransomware 폴더 내의 10개 폴더 경로 리스트 받아오기
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')

# 모든 파일 로드 및 샘플 개수 확인
for folder in ransomware_folder_list:
    ransomware_class = folder.split('\\')[-1]  # 랜섬웨어 클래스명 추출

    file_paths = glob.glob(folder + '\\*.csv')
    total_samples = 0  # 클래스별 전체 샘플 개수 초기화
    for file_path in file_paths:
        data = pd.read_csv(file_path)
        total_samples += data.shape[0]  # 데이터프레임의 행 수 / 샘플 개수 더하기

    samples_per_class[ransomware_class] = total_samples

# 클래스별 샘플 개수 출력
for ransomware_class, num_samples in samples_per_class.items():
    print(f"{ransomware_class} 클래스의 샘플 개수: {num_samples}")

Charger 클래스의 샘플 개수: 39551
Jisut 클래스의 샘플 개수: 25672
Koler 클래스의 샘플 개수: 44555
Lockerpin 클래스의 샘플 개수: 25307
Pletor 클래스의 샘플 개수: 4715
PornDroid 클래스의 샘플 개수: 46082
RansomBO 클래스의 샘플 개수: 39859
Simplocker 클래스의 샘플 개수: 36340
SVpeng 클래스의 샘플 개수: 54161
WannaLocker 클래스의 샘플 개수: 32701


In [3]:
# 각 랜섬웨어 클래스별로 4,000개씩 무작위 샘플 추출 (비복원 추출)
num_samples_per_ransomware_class = 4000
train_ransomware_data = []

# Ransomware 폴더 내의 10개 폴더 경로 리스트 받아오기
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')

for ransomware_folder_path in ransomware_folder_list:
    ransomware_file_paths = glob.glob(ransomware_folder_path + '/*.csv')
    ransomware_data = pd.DataFrame()

    # 각 랜섬웨어 클래스의 모든 파일 로드
    for file_path in ransomware_file_paths:
        data = pd.read_csv(file_path)
        ransomware_data = pd.concat([ransomware_data, data], ignore_index=True)

    # 랜섬웨어 클래스에서 4,000개 무작위 샘플 추출 (비복원 추출)
    if len(ransomware_data) > num_samples_per_ransomware_class:
        ransomware_subset = ransomware_data.sample(n=num_samples_per_ransomware_class, random_state=42, replace=False)
    else:
        ransomware_subset = ransomware_data.copy()
    train_ransomware_data.append(ransomware_subset)

# 모든 랜섬웨어 클래스의 데이터프레임들을 합치기
train_ransomware_data_subset = pd.concat(train_ransomware_data, ignore_index=True)

print("train_ransomware_subset의 샘플 개수:", len(train_ransomware_data_subset))

train_ransomware_subset의 샘플 개수: 40000


In [4]:
# 타깃 변수 열 이름 확인
target_variable = ' Label'  # 실제 타깃 변수 열 이름으로 수정

# X와 y로 데이터 분할
y = train_ransomware_data_subset[target_variable]  # 타깃 변수
X = train_ransomware_data_subset.drop([target_variable], axis=1)  # 타깃 변수 제외한 나머지 특성

# 불필요한 특성 제거
unnecessary_features = ['Flow ID', ' Timestamp', ' Source IP', ' Destination IP']
X = X.drop(unnecessary_features, axis=1)

# NaN 값을 0으로 대체
X = X.fillna(0)

# 0~255 범위로 스케일링
scaler = MinMaxScaler((0, 255))
X_scaled_8bit = pd.DataFrame(scaler.fit_transform(X), columns=X.columns).astype('uint8')

# 결과 출력
print(X_scaled_8bit)

        Source Port   Destination Port   Protocol   Flow Duration  \
0               236                156        255               0   
1               202                  0         90               1   
2               135                  0         90               0   
3               119                  0        255               0   
4                67                  0        255               1   
...             ...                ...        ...             ...   
39995           185                  1         90              61   
39996           168                  1         90               0   
39997           164                  1         90               0   
39998           173                  1         90               0   
39999           158                  1         90              55   

        Total Fwd Packets   Total Backward Packets  \
0                       0                        0   
1                       0                        0   
2        

In [6]:
# 각 행의 마지막에 0 값을 추가합니다.
X_padded = np.pad(X_scaled_8bit.values, ((0, 0), (0, 1)), mode='constant', constant_values=0)

# 9x9로 리쉐이핑합니다.
num_rows = 9
num_columns = 9
X_reshaped = X_padded.reshape(-1, num_rows, num_columns)

# 결과 출력
print("9x9 크기의 2차원 행렬:")
print(X_reshaped[0])  # 첫 번째 샘플에 해당하는 9x9 행렬 출력

9x9 크기의 2차원 행렬:
[[236 156 255   0   0   0   0   0   0]
 [  3   1   0   1  17  17   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [ 97 254   0   0   4   1   9   4   0]
 [  0   0   0   0   0   0   0   0   9]
 [ 10   1  17  97   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  84]
 [  0   0   0   0   0   0   0   0   0]]


In [12]:
from PIL import Image
import os
import numpy as np

# 이미지를 저장할 폴더를 생성합니다.
save_folder = r'C:\Users\pc\Desktop\CNN\Multiclass_All_Train_Image'

# 각 랜섬웨어 클래스 폴더를 생성합니다.
ransomware_classes = ["Charger", "Jisut", "Koler", "Lockerpin", "Pletor", "PornDroid", "RansomBO", "Simplocker", "SVpeng", "WannaLocker"]
for ransomware_class in ransomware_classes:
    os.makedirs(os.path.join(save_folder, ransomware_class), exist_ok=True)

# X_final_reshaped를 이미지로 변환하여 저장합니다.
num_samples = X_reshaped.shape[0]
resize_factor = 10

for i in range(num_samples):
    # 8비트 그레이스케일 이미지 생성
    image_data = X_reshaped[i]
    image = Image.fromarray(image_data, 'L')  # 'L'은 8비트 그레이스케일 이미지를 나타냅니다.
    
    # 이미지 데이터 확대
    image_data = np.repeat(image_data, resize_factor, axis=0)  # 행 확대
    image_data = np.repeat(image_data, resize_factor, axis=1)  # 열 확대

    # 확대된 데이터를 다시 이미지로 변환
    image = Image.fromarray(image_data, 'L')  # 8비트 그레이스케일 이미지로 변환
    
    # 이미지의 라벨에 따라서 저장 위치 및 파일명 설정
    if i < 4000:
        label = "Charger"
    elif 4000 <= i < 8000:
        label = "Jisut"
    elif 8000 <= i < 12000:
        label = "Koler"
    elif 12000 <= i < 16000:
        label = "Lockerpin"
    elif 16000 <= i < 20000:
        label = "Pletor"
    elif 20000 <= i < 24000:
        label = "PornDroid"
    elif 24000 <= i < 28000:
        label = "RansomBO"
    elif 28000 <= i < 32000:
        label = "Simplocker"
    elif 32000 <= i < 36000:
        label = "SVpeng"
    else:
        label = "WannaLocker"

    # 이미지 저장
    image_filename = f"image_{i}.png"
    image.save(os.path.join(save_folder, label, image_filename))

print("이미지 변환 및 저장이 완료되었습니다.")

이미지 변환 및 저장이 완료되었습니다.


In [13]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical

# 이미지 데이터를 저장한 폴더 경로
image_folder = r'C:\Users\pc\Desktop\CNN\Multiclass_All_Train_Image\\'

# 이미지 데이터를 불러오고 라벨을 지정합니다.
X_data = []
y_labels = []

label_names = ["Charger", "Jisut", "Koler", "Lockerpin", "Pletor", "PornDroid", "RansomBO", "Simplocker", "SVpeng", "WannaLocker"]


for i in range(num_samples):
    if i < 4000:
        label_index = 0           # Charger
    elif 4000 <= i < 8000:
        label_index = 1           # Jisut 
    elif 8000 <= i < 12000:
        label_index = 2           # Koler
    elif 12000 <= i < 16000:
        label_index = 3           # Lockerpin
    elif 16000 <= i < 20000:
        label_index = 4           # Pletor
    elif 20000 <= i < 24000:
        label_index = 5           # PornDroid
    elif 24000 <= i < 28000:
        label_index = 6           # RansomBO
    elif 28000 <= i < 32000:
        label_index = 7           # Simplocker
    elif 32000 <= i < 36000:
        label_index = 8           # SVpeng
    else:
        label_index = 9           # WannaLocker

    label_name = label_names[label_index]
    image_path = os.path.join(image_folder, label_name, f"image_{i}.png")
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    X_data.append(image)
    y_labels.append(label_index)

X_data = np.array(X_data)
y_labels = np.array(y_labels)

y_onehot = to_categorical(y_labels, num_classes=len(label_names))

In [15]:
# 새로 추출할 샘플 개수 설정
num_new_samples_per_ransomware_class = 500

# 샘플을 저장할 리스트
val_ransomware_subset = []

for ransomware_folder_path in ransomware_folder_list:
    ransomware_file_paths = glob.glob(ransomware_folder_path + '/*.csv')
    ransomware_data = pd.DataFrame()

    # 각 랜섬웨어 클래스의 모든 파일 로드
    for file_path in ransomware_file_paths:
        data = pd.read_csv(file_path)
        ransomware_data = pd.concat([ransomware_data, data], ignore_index=True)
    
    # 샘플에서 500개씩 추출
    if len(ransomware_data) > num_new_samples_per_ransomware_class:
        ransomware_subset = ransomware_data.sample(n=num_new_samples_per_ransomware_class, random_state=42, replace=False)
    else:
        ransomware_subset = ransomware_data.copy()
    val_ransomware_subset.append(ransomware_subset)

# 모든 랜섬웨어 클래스의 샘플 데이터프레임들을 합치기
val_ransomware_subset_add = pd.concat(val_ransomware_subset, ignore_index=True)

print("Validation_ransomware_subset의 샘플 개수:", len(val_ransomware_subset_add))


Validation_ransomware_subset의 샘플 개수: 5000


In [16]:
# 타깃 변수 열 이름 확인
target_variable = ' Label'  # 실제 타깃 변수 열 이름으로 수정

# X와 y로 데이터 분할
y = val_ransomware_subset_add[target_variable]  # 타깃 변수
X = val_ransomware_subset_add.drop([target_variable], axis=1)  # 타깃 변수 제외한 나머지 특성

# 불필요한 특성 제거
unnecessary_features = ['Flow ID', ' Timestamp', ' Source IP', ' Destination IP']
X = X.drop(unnecessary_features, axis=1)

# NaN 값을 0으로 대체
X = X.fillna(0)

# 0~255 범위로 스케일링
scaler = MinMaxScaler((0, 255))
Val_X_scaled_8bit = pd.DataFrame(scaler.fit_transform(X), columns=X.columns).astype('uint8')

# 결과 출력
print(Val_X_scaled_8bit)

       Source Port   Destination Port   Protocol   Flow Duration  \
0              236                157        255               0   
1              202                  0         90               1   
2              135                  0         90               0   
3              119                  0        255               0   
4               67                  0        255               1   
...            ...                ...        ...             ...   
4995           140                  1         90             123   
4996           141                  0         90              44   
4997           236                  0        255               1   
4998           232                  1         90              64   
4999           161                  1         90              39   

       Total Fwd Packets   Total Backward Packets  \
0                      0                        0   
1                      0                        0   
2                      0

In [17]:
# 각 행의 마지막에 0 값을 추가합니다.
X_padded = np.pad(Val_X_scaled_8bit.values, ((0, 0), (0, 1)), mode='constant', constant_values=0)

# 9x9로 리쉐이핑합니다.
Val_X_reshaped = X_padded.reshape(-1, num_rows, num_columns)

# 결과 출력
print("9x9 크기의 2차원 행렬:")
print(Val_X_reshaped[0])  # 첫 번째 샘플에 해당하는 9x9 행렬 출력

9x9 크기의 2차원 행렬:
[[236 157 255   0   0   0   0   0   0]
 [  3   3   0   2  17  17   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  17   1  10   6   0]
 [  0   0   0   0   0   0   0   0  31]
 [ 15   3  17   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  47]
 [  0   0   0   0   0   0   0   0   0]]


In [26]:
from PIL import Image
import os
import numpy as np

# 이미지를 저장할 폴더를 생성합니다.
save_folder = r'C:\Users\pc\Desktop\CNN\Multiclass_All_Validation_Image\\'

# 각 랜섬웨어 클래스 폴더를 생성합니다.
ransomware_classes = ["Charger", "Jisut", "Koler", "Lockerpin", "Pletor", "PornDroid", "RansomBO", "Simplocker", "SVpeng", "WannaLocker"]
for ransomware_class in ransomware_classes:
    os.makedirs(os.path.join(save_folder, ransomware_class), exist_ok=True)

# X_final_reshaped를 이미지로 변환하여 저장합니다.
num_samples = Val_X_reshaped.shape[0]
resize_factor = 10

for i in range(num_samples):
    # 8비트 그레이스케일 이미지 생성
    image_data = Val_X_reshaped[i]
    image = Image.fromarray(image_data, 'L')  # 'L'은 8비트 그레이스케일 이미지를 나타냅니다.
    
    # 이미지 데이터 확대
    image_data = np.repeat(image_data, resize_factor, axis=0)  # 행 확대
    image_data = np.repeat(image_data, resize_factor, axis=1)  # 열 확대

    # 확대된 데이터를 다시 이미지로 변환
    image = Image.fromarray(image_data, 'L')  # 8비트 그레이스케일 이미지로 변환
    
    # 이미지의 라벨에 따라서 저장 위치 및 파일명 설정
    if i < 500:
        label = "Charger"
    elif 500 <= i < 1000:
        label = "Jisut"
    elif 1000 <= i < 1500:
        label = "Koler"
    elif 1500 <= i < 2000:
        label = "Lockerpin"
    elif 2000 <= i < 2500:
        label = "Pletor"
    elif 2500 <= i < 3000:
        label = "PornDroid"
    elif 3000 <= i < 3500:
        label = "RansomBO"
    elif 3500 <= i < 4000:
        label = "Simplocker"
    elif 4000 <= i < 4500:
        label = "SVpeng"
    else:
        label = "WannaLocker"

    # 이미지 저장
    image_filename = f"image_{i}.png"
    image.save(os.path.join(save_folder, label, image_filename))

print("이미지 변환 및 저장이 완료되었습니다.")

이미지 변환 및 저장이 완료되었습니다.


In [27]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical

# 이미지 데이터를 저장한 폴더 경로
image_folder = r'C:\Users\pc\Desktop\CNN\Multiclass_All_Validation_Image\\'

# 이미지 데이터를 불러오고 라벨을 지정합니다.
X_data = []
y_labels = []

label_names = ["Charger", "Jisut", "Koler", "Lockerpin", "Pletor", "PornDroid", "RansomBO", "Simplocker", "SVpeng", "WannaLocker"]


for i in range(num_samples):
    if i < 500:
        label_index = 0           # Charger
    elif 500 <= i < 1000:
        label_index = 1           # Jisut 
    elif 1000 <= i < 1500:
        label_index = 2           # Koler
    elif 1500 <= i < 2000:
        label_index = 3           # Lockerpin
    elif 2000 <= i < 2500:
        label_index = 4           # Pletor
    elif 2500 <= i < 3000:
        label_index = 5           # PornDroid
    elif 3000 <= i < 3500:
        label_index = 6           # RansomBO
    elif 3500 <= i < 4000:
        label_index = 7           # Simplocker
    elif 4000 <= i < 4500:
        label_index = 8           # SVpeng
    else:
        label_index = 9           # WannaLocker

    label_name = label_names[label_index]
    image_path = os.path.join(image_folder, label_name, f"image_{i}.png")
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    X_data.append(image)
    y_labels.append(label_index)

X_data = np.array(X_data)
y_labels = np.array(y_labels)

y_onehot = to_categorical(y_labels, num_classes=len(label_names))

In [28]:
# 각 랜섬웨어 클래스별로 500개씩 무작위 샘플 추출 (비복원 추출)
num_samples_per_ransomware_class = 500
test_ransomware_data = []

# Ransomware 폴더 내의 10개 폴더 경로 리스트 받아오기
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')

for ransomware_folder_path in ransomware_folder_list:
    ransomware_file_paths = glob.glob(ransomware_folder_path + '/*.csv')
    ransomware_data = pd.DataFrame()

    # 각 랜섬웨어 클래스의 모든 파일 로드
    for file_path in ransomware_file_paths:
        data = pd.read_csv(file_path)
        ransomware_data = pd.concat([ransomware_data, data], ignore_index=True)

    # 랜섬웨어 클래스에서 500개 무작위 샘플 추출 (비복원 추출)
    if len(ransomware_data) > num_samples_per_ransomware_class:
        ransomware_subset = ransomware_data.sample(n=num_samples_per_ransomware_class, random_state=42, replace=False)
    else:
        ransomware_subset = ransomware_data.copy()
    test_ransomware_data.append(ransomware_subset)

# 모든 랜섬웨어 클래스의 데이터프레임들을 합치기
test_ransomware_data = pd.concat(test_ransomware_data, ignore_index=True)

print("test_ransomware_subset의 샘플 개수:", len(test_ransomware_data))

test_ransomware_subset의 샘플 개수: 5000


In [29]:
# 타깃 변수 열 이름 확인
target_variable = ' Label'  # 실제 타깃 변수 열 이름으로 수정

# X와 y로 데이터 분할
y = test_ransomware_data[target_variable]  # 타깃 변수
X = test_ransomware_data.drop([target_variable], axis=1)  # 타깃 변수 제외한 나머지 특성

# 불필요한 특성 제거
unnecessary_features = ['Flow ID', ' Timestamp', ' Source IP', ' Destination IP']
X = X.drop(unnecessary_features, axis=1)

# NaN 값을 0으로 대체
X = X.fillna(0)

# 0~255 범위로 스케일링
scaler = MinMaxScaler((0, 255))
Test_X_scaled_8bit = pd.DataFrame(scaler.fit_transform(X), columns=X.columns).astype('uint8')

# 결과 출력
print(Test_X_scaled_8bit)

       Source Port   Destination Port   Protocol   Flow Duration  \
0              236                157        255               0   
1              202                  0         90               1   
2              135                  0         90               0   
3              119                  0        255               0   
4               67                  0        255               1   
...            ...                ...        ...             ...   
4995           140                  1         90             123   
4996           141                  0         90              44   
4997           236                  0        255               1   
4998           232                  1         90              64   
4999           161                  1         90              39   

       Total Fwd Packets   Total Backward Packets  \
0                      0                        0   
1                      0                        0   
2                      0

In [30]:
# 각 행의 마지막에 0 값을 추가합니다.
X_padded = np.pad(Test_X_scaled_8bit.values, ((0, 0), (0, 1)), mode='constant', constant_values=0)

# 9x9로 리쉐이핑합니다.
Test_X_reshaped = X_padded.reshape(-1, num_rows, num_columns)

# 결과 출력
print("9x9 크기의 2차원 행렬:")
print(Test_X_reshaped[0])  # 첫 번째 샘플에 해당하는 9x9 행렬 출력

9x9 크기의 2차원 행렬:
[[236 157 255   0   0   0   0   0   0]
 [  3   3   0   2  17  17   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  17   1  10   6   0]
 [  0   0   0   0   0   0   0   0  31]
 [ 15   3  17   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  47]
 [  0   0   0   0   0   0   0   0   0]]


In [32]:
from PIL import Image
import os
import numpy as np

# 이미지를 저장할 폴더를 생성합니다.
save_folder = r'C:\Users\pc\Desktop\CNN\Multiclass_All_Test_Image\\'

# 각 랜섬웨어 클래스 폴더를 생성합니다.
ransomware_classes = ["Charger", "Jisut", "Koler", "Lockerpin", "Pletor", "PornDroid", "RansomBO", "Simplocker", "SVpeng", "WannaLocker"]
for ransomware_class in ransomware_classes:
    os.makedirs(os.path.join(save_folder, ransomware_class), exist_ok=True)

# X_final_reshaped를 이미지로 변환하여 저장합니다.
num_samples = Test_X_reshaped.shape[0]
resize_factor = 10

for i in range(num_samples):
    # 8비트 그레이스케일 이미지 생성
    image_data = Test_X_reshaped[i]
    image = Image.fromarray(image_data, 'L')  # 'L'은 8비트 그레이스케일 이미지를 나타냅니다.
    
    # 이미지 데이터 확대
    image_data = np.repeat(image_data, resize_factor, axis=0)  # 행 확대
    image_data = np.repeat(image_data, resize_factor, axis=1)  # 열 확대

    # 확대된 데이터를 다시 이미지로 변환
    image = Image.fromarray(image_data, 'L')  # 8비트 그레이스케일 이미지로 변환
    
    # 이미지의 라벨에 따라서 저장 위치 및 파일명 설정
    if i < 500:
        label = "Charger"
    elif 500 <= i < 1000:
        label = "Jisut"
    elif 1000 <= i < 1500:
        label = "Koler"
    elif 1500 <= i < 2000:
        label = "Lockerpin"
    elif 2000 <= i < 2500:
        label = "Pletor"
    elif 2500 <= i < 3000:
        label = "PornDroid"
    elif 3000 <= i < 3500:
        label = "RansomBO"
    elif 3500 <= i < 4000:
        label = "Simplocker"
    elif 4000 <= i < 4500:
        label = "SVpeng"
    else:
        label = "WannaLocker"

    # 이미지 저장
    image_filename = f"image_{i}.png"
    image.save(os.path.join(save_folder, label, image_filename))

print("이미지 변환 및 저장이 완료되었습니다.")

이미지 변환 및 저장이 완료되었습니다.


In [33]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical

# 이미지 데이터를 저장한 폴더 경로
image_folder = r'C:\Users\pc\Desktop\CNN\Multiclass_All_Test_Image\\'

# 이미지 데이터를 불러오고 라벨을 지정합니다.
X_data = []
y_labels = []

label_names = ["Charger", "Jisut", "Koler", "Lockerpin", "Pletor", "PornDroid", "RansomBO", "Simplocker", "SVpeng", "WannaLocker"]


for i in range(num_samples):
    if i < 500:
        label_index = 0           # Charger
    elif 500 <= i < 1000:
        label_index = 1           # Jisut 
    elif 1000 <= i < 1500:
        label_index = 2           # Koler
    elif 1500 <= i < 2000:
        label_index = 3           # Lockerpin
    elif 2000 <= i < 2500:
        label_index = 4           # Pletor
    elif 2500 <= i < 3000:
        label_index = 5           # PornDroid
    elif 3000 <= i < 3500:
        label_index = 6           # RansomBO
    elif 3500 <= i < 4000:
        label_index = 7           # Simplocker
    elif 4000 <= i < 4500:
        label_index = 8           # SVpeng
    else:
        label_index = 9           # WannaLocker

    label_name = label_names[label_index]
    image_path = os.path.join(image_folder, label_name, f"image_{i}.png")
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    X_data.append(image)
    y_labels.append(label_index)

X_data = np.array(X_data)
y_labels = np.array(y_labels)

y_onehot = to_categorical(y_labels, num_classes=len(label_names))